In [ ]:
! pip --upgrade --quiet install arize-phoenix[evals]

In [ ]:
#invoke tracing via phoenix
from phoenix.trace.langchain import LangChainInstrumentor

LangChainInstrumentor().instrument()

In [57]:
import weaviate

client = weaviate.Client("http://localhost:8081")
weaviate_client = weaviate.connect_to_local("localhost","8081")#v4
index_name="Phoenix_test"

In [68]:
weaviate_client.collections.delete_all()
weaviate_client.collections.list_all()

{}

In [ ]:
from weaviate.classes.config import Configure
from weaviate.classes.config import Property, DataType

weaviate_client.collections.create(
    index_name,
    #see notes above re: the docker modules that need to be enabled for text2vec* to work correctly -e ENABLE_MODULES=text2vec-ollama
    vectorizer_config=Configure.Vectorizer.text2vec_ollama( 
        model="nomic-embed-text",    
        api_endpoint="http://host.docker.internal:11434",
    ),
    # generative_config=Configure.Generative.ollama(
    #     api_endpoint = "http://host.docker.internal:11434",
    #     model="jonphi"
    # ),

    # properties=[
    #     Property(name="page_content", data_type=DataType.TEXT),
    #     Property(name="source", data_type=DataType.INT),
    # ]

)

In [71]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
import pprint

loader = TextLoader("/home/jonot480/Documents/Coke OCG.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
docs = text_splitter.split_documents(documents)
#print(*docs, sep='\n')

In [72]:
from langchain_community.vectorstores.weaviate import Weaviate
from langchain_community.embeddings.ollama import OllamaEmbeddings

weav = Weaviate.from_documents(
    docs, 
    embedding=OllamaEmbeddings(model='nomic-embed-text'),
    client=client,    
    index_name=index_name,
    #prefer_grpc=True, 
) 

In [ ]:
collection = weaviate_client.collections.get(index_name)
response = collection.query.fetch_objects()

for o in response.objects:
    print(o.properties)  # Inspect returned objects
    #print(o.properties["text"])


In [74]:

disbs = weav.similarity_search("effective date",k=4)
print(disbs)

[Document(page_content='Unless agreed otherwise with Contact Counsel, Outside Counsel shall submit monthly invoices for\nlegal services provided to Coca-Cola HBC if the statements exceed Euro 1,000 (or the converted\namount in the currency of the country of the respective Coca-Cola HBC subsidiary).\nOtherwise, invoices should be rendered quarterly or upon completion of the matter within the\nquarterly period, whichever is earlier. Invoices should be received within 30 days of the earlier of\nthe end of the billing period or completion of the matter.\nThe timely closing of Coca-Cola HBC’s books at year-end requires the prompt submission of year-\nend invoices. Firms which bill on a quarterly basis should submit one statement for the months of\nOctober and November to be received no later than December 20. For accrual purposes, Outside\nCounsel shall calculate anticipated fees and disbursements for December and shall provide\nContact Counsel with an estimate of these charges by December 

In [ ]:

backup = weav.similarity_search("invoice copy",k=10)
print(backup)

In [ ]:
rates = weav.similarity_search("hourly rates",k=10)

print(rates)

In [ ]:
accruals = weav.similarity_search("accrual",k=10)

print(accruals)

In [76]:
from langchain_community.retrievers import (
    WeaviateHybridSearchRetriever,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="jonphi")

retriever = WeaviateHybridSearchRetriever(
    client=client,
    index_name=index_name,
    text_key="text",
    attributes=[],
    create_schema_if_missing=True,
    k=4
)

query="effective date"

result = retriever.invoke(query,score=True)
        
after_rag_template="""
<context>
{context}
</context>
Question: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain=(
    {"context":retriever,"question":RunnablePassthrough()}
    | after_rag_prompt
    | llm
    | StrOutputParser()    
)
print(after_rag_chain.invoke("what is the effective date?"))


The effective date is November 1, 2020.
